In [ ]:
!pip install pandas
!pip install transformers
!pip install datasets
!pip install torch
!pip install numpy
!pip install nltk

import pandas as pd
from transformers import AutoTokenizer
from transformers import TFAutoModel
from transformers import TFAutoModelForSequenceClassification
from datasets import load_dataset
from datasets import *
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from transformers import AutoConfig
import torch
import numpy as np
from transformers import Trainer
import nltk



In [ ]:

testSet = pd.read_csv('0_Tag.csv')

bert_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [ ]:

#bert_model_name = "bert-base-uncased"
bert_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)



def align_labels(examples):
    tokenized_inputs = bert_tokenizer(examples["Text"], truncation=True,
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["Label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["Label"] = labels
    return tokenized_inputs

In [28]:
tokensSet=[]
for row in testSet['Text']:
    tokensSet.append(bert_tokenizer(row).tokens())

testSet['tokens']=tokensSet

Read and tokenize some files

In [ ]:

nltk.tokenize.sent_tokenize(text_body, language='english')

NameError: name 'text_body' is not defined

In [ ]:
#theFiles=["0_Tag.csv","1_Tag.csv","2_Tag.csv","3_Tag.csv","4_Tag.csv","5_Tag.csv","6_Tag.csv"]

for i in range(0,14):
    print(i)
    testSet = pd.read_csv(str(i) + '_Tag.csv')
    bertSet=[]
    bert_model_name = "xlm-roberta-base"
    bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
    tokensSet=[]
    for row in testSet['Text']:
        #print(row)
        tokensSet.append(bert_tokenizer(row).tokens())
        bertSet.append(bert_tokenizer(row))

    testSet['tokens']=tokensSet
    testSet['berts']=bertSet
    testSet.to_csv(str(i) + '_Tag_Tokenize.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [ ]:

model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
#print(tokenizer.model_max_length)

def tokenize(batch):
    return tokenizer(batch['Text'], padding=True, truncation=True, max_length=tokenizer.model_max_length)


#theDS_encoded = testSet.map(tokenize)

Read in the files to a dataset for easier interpretation.  These have been manually tagged.

In [121]:
datafiles=[]
for i in range(0,14):
    datafiles.append(str(i) + '_Tag.csv')

alldata=pd.DataFrame()
for i in datafiles:
    allData=pd.concat([alldata,pd.read_csv(i).T])



In [ ]:


class_names = ['O', 'PRESENT','ABSENT','VOTE','PROJ']

ft = Features({'Text': Value('string'), 'Label': ClassLabel(num_classes=5,names=class_names)})
testDataset = load_dataset("csv", 'testing_PD.csv',)
testDataset

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'Text', 'Label'],
        num_rows: 1
    })
})

In [122]:
text1=pd.read_csv('0_Tag_Trim.csv')
#str(x) for x in xs
exText = [' '.join(text1['Text'].to_list())]
exLabel = [','.join(str(x) for x in text1['Label'].to_list())]

In [ ]:
testingPD=pd.DataFrame()
testingPD['Text'] = exText
testingPD['Label'] = exLabel

#testingPD.to_csv('testing_PD.csv')

Starting another group to work through class 12 file.

In [ ]:
# hide_output


bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [230]:
bert_tokens = bert_tokenizer(exText[0]).tokens()
xlmr_tokens = xlmr_tokenizer(exText[0]).tokens()

Token indices sequence length is longer than the specified maximum sequence length for this model (5691 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5956 > 512). Running this sequence through the model will result in indexing errors


In [231]:
df = pd.DataFrame([bert_tokens, xlmr_tokens], index=["BERT", "XLM-R"])
df

,0,1,2,3,4,5,6,7,8,9,...,5946,5947,5948,5949,5950,5951,5952,5953,5954,5955
BERT,[CLS],Hall,",",with,Co,-,Chair,##person,",",Jean,...,None,None,None,None,None,None,None,None,None,None
XLM-R,<s>,▁Hall,",",▁with,▁Co,▁-,Cha,ir,person,",",...,▁Plan,ning,▁Board,▁Secretary,▁Plan,ning,▁Technic,ia,▁n,</s>


In [ ]:


class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        # we set add_​pool⁠ing_layer=False to ensure all hidden states are returned and not only the one associated with the [CLS] token.
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                labels=None):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids)
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits,
                                     hidden_states=outputs.hidden_states,
                                     attentions=outputs.attentions)

In [234]:
index2tag = {idx: tag for idx, tag in enumerate(class_names)}
tag2index = {tag: idx for idx, tag in enumerate(class_names)}

In [ ]:


xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,
                                         num_labels=len(class_names),
                                         id2label=index2tag,
                                         label2id=tag2index)

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [265]:
from nltk.tokenize import sent_tokenize
import nltk
sent_text = nltk.sent_tokenize(exText[0])

In [284]:
len(exText[0].split(' '))

4349

In [286]:
len(exLabel[0].split(','))

4349

In [288]:
count=0
for i in sent_text:
    count+=len(i.split(' '))
print(count)

4349


In [356]:
exTextSplit=exText[0].split(' ')
theDF = pd.DataFrame([exText[0].split(' '),exLabel[0].split(',')],index=["Tokens", "Labels"])
theDF
theDFT=theDF.T

In [357]:
dfDS = Dataset.from_pandas(theDFT)

dfDS

Dataset({
    features: ['Tokens', 'Labels'],
    num_rows: 4349
})

In [393]:
train_testvalid = dfDS.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [394]:
ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Labels'],
        num_rows: 3479
    })
    test: Dataset({
        features: ['Tokens', 'Labels'],
        num_rows: 435
    })
    validation: Dataset({
        features: ['Tokens', 'Labels'],
        num_rows: 435
    })
})

In [388]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["Tokens"], truncation=True,
                                      is_split_into_words=False)
    labels = []
    for label in enumerate(examples["Labels"]):
        word_ids = tokenized_inputs.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels,batched=True,
                        remove_columns=['Tokens','Labels'])

In [395]:
encodedDS = encode_panx_dataset(ds)
encodedDS

Map: 100%|██████████| 435/435 [00:00<00:00, 63765.50 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3479
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 435
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 435
    })
})

In [398]:
#encodedDS['input_ids']

In [ ]:


def align_predictions(predictions, label_ids):
    # Function to take a set of predictions and align them with a list of labels to use in the classification report
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [365]:
from transformers import TrainingArguments

num_epochs = 15
batch_size = 24
logging_steps = len(ds["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-en"
training_args = TrainingArguments(output_dir=model_name,
                                  log_level="error",
                                  num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  eval_strategy="epoch",
                                  save_steps=1e6,
                                  weight_decay=0.01,
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  report_to='none')

In [366]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [367]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [368]:
def model_init():
    return (XLMRobertaForTokenClassification
            .from_pretrained(xlmr_model_name, config=xlmr_config)
            .to(device))

In [392]:
encodedDS

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3479
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 435
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 435
    })
})

In [ ]:

trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=encodedDS["train"],
                  eval_dataset=encodedDS["validation"],
                  processing_class=xlmr_tokenizer)

In [397]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 